# HW3 Image Classification
## We strongly recommend that you run with [Kaggle](https://www.kaggle.com/t/86ca241732c04da99aca6490080bae73) for this homework

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Check GPU Type

In [39]:
!nvidia-smi

Thu Mar 23 00:55:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.78.01    Driver Version: 525.78.01    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   49C    P2    54W / 300W |  10336MiB / 11264MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0  On |                  N/A |
| 22%   

# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.


In [40]:
# Download Link
# Link 1 (Dropbox): https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0
# Link 2 (Google Drive): https://drive.google.com/file/d/1tbGNwk1yGoCBdu4Gi_Cia7EJ9OhubYD9/view?usp=share_link
# Link 3: Kaggle Competition.

# (1) dropbox link
#!wget -O food11.zip https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0

# (2) google drive link
#!gdown --id '1tbGNwk1yGoCBdu4Gi_Cia7EJ9OhubYD9' --output food11.zip

In [41]:
#! unzip food11.zip

# Import Packages

In [1]:
_exp_name = "sample"

In [2]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from torchvision.models import resnet18, resnext50_32x4d

In [3]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Transforms
Torchvision provides lots of useful utilities for image preprocessing, data *wrapping* as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [4]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.TenCrop((224, 224)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    #transforms.Resize((128, 128)),
    # You may add some transforms here.
    transforms.RandomResizedCrop((224, 224)),
    transforms.ColorJitter(),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# test_tfm_cj = transforms.Compose([
#     # Resize the image into a fixed shape (height = width = 128)
#     transforms.Resize((128, 128)),
#     # You may add some transforms here.
#     transforms.ColorJitter(),
    
#     # ToTensor() should be the last one of the transforms.
#     transforms.ToTensor(),
# ])

# test_tfm_rr = transforms.Compose([
#     # Resize the image into a fixed shape (height = width = 128)
#     transforms.Resize((128, 128)),
#     # You may add some transforms here.
#     transforms.RandomRotation(30),
    
#     # ToTensor() should be the last one of the transforms.
#     transforms.ToTensor(),
# ])

# test_tfm_rhf = transforms.Compose([
#     # Resize the image into a fixed shape (height = width = 128)
#     transforms.Resize((128, 128)),
#     # You may add some transforms here.
#     transforms.RandomHorizontalFlip(),
    
#     # ToTensor() should be the last one of the transforms.
#     transforms.ToTensor(),
# ])

# test_tfm_rrc = transforms.Compose([
#     # Resize the image into a fixed shape (height = width = 128)
#     transforms.Resize((128, 128)),
#     # You may add some transforms here.
#     transforms.RandomResizedCrop((128, 128)),
    
#     # ToTensor() should be the last one of the transforms.
#     transforms.ToTensor(),
# ])


# Datasets
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [5]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
            
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
            
        return im,label

# Model

In [47]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

# Configurations

In [6]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
#model = Classifier().to(device)
#model = resnet18(weights=None).to(device)
model = resnext50_32x4d(weights=None).to(device)

# The number of batch size.
batch_size = 64

# The number of training epochs.
n_epochs = 260

# If no improvement in 'patience' epochs, early stop.
patience = 50

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.AdamW(model.parameters(), lr=0.002, weight_decay=1e-3)

# Dataloader

In [49]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset("./train", tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset("./valid", tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

# Start Training

In [50]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-5)

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    #scheduler.step()
    # Print the information.
    #print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best_3.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

  0%|          | 0/157 [00:00<?, ?it/s]

[ Train | 001/300 ] loss = 2.71354, acc = 0.22761


  0%|          | 0/57 [00:00<?, ?it/s]

[ Valid | 001/300 ] loss = 2.81993, acc = 0.22620 -> best
Best model found at epoch 0, saving model


  0%|          | 0/157 [00:00<?, ?it/s]

...

[ Valid | 299/300 ] loss = 1.18703, acc = 0.84027


  0%|          | 0/157 [00:00<?, ?it/s]

[ Train | 300/300 ] loss = 0.68015, acc = 0.95810


  0%|          | 0/57 [00:00<?, ?it/s]

[ Valid | 300/300 ] loss = 1.12636, acc = 0.85183


# Dataloader for test

In [7]:
# Construct test datasets.
# The argument "loader" tells how torchvision reads the data.
# test_set = FoodDataset("./test", tfm=test_tfm)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# test_set2 = FoodDataset("./test", tfm=train_tfm)
# test_loader2 = DataLoader(test_set2, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# test_set3 = FoodDataset("./test", tfm=train_tfm)
# test_loader3 = DataLoader(test_set2, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# test_set_rrc = FoodDataset("./test", tfm=test_tfm_rrc)
# test_loader_rrc = DataLoader(test_set_rrc, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# test_set_rr = FoodDataset("./test", tfm=test_tfm_rr)
# test_loader_rr = DataLoader(test_set_rr, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# test_set_cj = FoodDataset("./test", tfm=test_tfm_cj)
# test_loader_cj = DataLoader(test_set_cj, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# test_set_rhf = FoodDataset("./test", tfm=test_tfm_rhf)
# test_loader_rhf = DataLoader(test_set_rhf, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# Testing and generate prediction CSV

In [8]:
# #model_best = Classifier().to(device)
# #model_best = resnet18(weights=None).to(device)
# model_best = resnext50_32x4d(weights=None).to(device)
# model_best.load_state_dict(torch.load(f"{_exp_name}_best_resnext_85588.ckpt"))
# model_best.eval()

# model_best2 = resnext50_32x4d(weights=None).to(device)
# model_best2.load_state_dict(torch.load(f"{_exp_name}_best_resnext_85231.ckpt"))
# model_best2.eval()

# model_best3 = resnext50_32x4d(weights=None).to(device)
# model_best3.load_state_dict(torch.load(f"{_exp_name}_best_resnext_85339.ckpt"))
# model_best3.eval()
# prediction = []
# with torch.no_grad():
#     #for (data,_), (data2,_), (data_rrc,_), (data_rr,_), (data_cj,_), (data_rhf,_) in zip(tqdm(test_loader), test_loader2, test_loader_rrc, test_loader_rr, test_loader_cj, test_loader_rhf):
#     for (data,_), (data2,_), (data3,_) in zip(tqdm(test_loader), test_loader2, test_loader3):
#         test_pred = model_best(data.to(device))
#         test_pred2 = model_best(data2.to(device))
#         test_pred3 = model_best(data3.to(device))

#         test2_pred = model_best2(data.to(device))
#         test2_pred2 = model_best2(data2.to(device))
#         test2_pred3 = model_best2(data3.to(device))

#         test3_pred = model_best3(data.to(device))
#         test3_pred2 = model_best3(data2.to(device))
#         test3_pred3 = model_best3(data3.to(device))

#         # test_pred_rrc = model_best(data_rrc.to(device))
#         # test_pred_rr = model_best(data_rr.to(device))
#         # test_pred_cj = model_best(data_cj.to(device))
#         # test_pred_rhf = model_best(data_rhf.to(device))

#         # final_pred = test_pred * 0.7 + test_pred2 * 0.1 + test_pred_rrc * 0.05 + test_pred_rr * 0.05 + test_pred_cj * 0.05 + test_pred_rhf * 0.05
#         final_pred = (test_pred * 0.65 + test_pred2 * 0.175 + test_pred3 * 0.175) * 0.34 \
#                    + (test2_pred * 0.65 + test2_pred2 * 0.175 + test2_pred3 * 0.175) * 0.33 \
#                    + (test3_pred * 0.65 + test3_pred2 * 0.175 + test3_pred3 * 0.175) * 0.33
#         # + (test2_pred * 0.65 + test2_pred2 * 0.175 + test2_pred3 * 0.175) * 0.33 \
#         #test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
#         final_label = np.argmax(final_pred.cpu().data.numpy(), axis=1)
#         #print(test_label)

#         prediction += final_label.squeeze().tolist()

  0%|          | 0/47 [00:00<?, ?it/s]

In [9]:
# # create test csv
# def pad4(i):
#     return "0"*(4-len(str(i)))+str(i)
# df = pd.DataFrame()
# df["Id"] = [pad4(i) for i in range(len(test_set))]
# df["Category"] = prediction
# df.to_csv("submission.csv",index = False)

# Q1. Augmentation Implementation
## Implement augmentation by finishing train_tfm in the code with image size of your choice. 
## Directly copy the following block and paste it on GradeScope after you finish the code
### Your train_tfm must be capable of producing 5+ different results when given an identical image multiple times.
### Your  train_tfm in the report can be different from train_tfm in your training code.


In [33]:
# train_tfm = transforms.Compose([
#     # Resize the image into a fixed shape (height = width = 128)
#     transforms.Resize((128, 128)),
#     # You can add some transforms here.
#     transforms.ToTensor(),
# ])

# Q2. Visual Representations Implementation
## Visualize the learned visual representations of the CNN model on the validation set by implementing t-SNE (t-distributed Stochastic Neighbor Embedding) on the output of both top & mid layers (You need to submit 2 images). 


In [34]:
# import torch
# import numpy as np
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import matplotlib.cm as cm
# import torch.nn as nn

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # Load the trained model
# model = Classifier().to(device)
# state_dict = torch.load(f"{_exp_name}_best.ckpt")
# model.load_state_dict(state_dict)
# model.eval()

# print(model)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# # Load the vaildation set defined by TA
# valid_set = FoodDataset("./valid", tfm=test_tfm)
# valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

# # Extract the representations for the specific layer of model
# index = ... # You should find out the index of layer which is defined as "top" or 'mid' layer of your model.
# features = []
# labels = []
# for batch in tqdm(valid_loader):
#     imgs, lbls = batch
#     with torch.no_grad():
#         logits = model.cnn[:index](imgs.to(device))
#         logits = logits.view(logits.size()[0], -1)
#     labels.extend(lbls.cpu().numpy())
#     logits = np.squeeze(logits.cpu().numpy())
#     features.extend(logits)
    
# features = np.array(features)
# colors_per_class = cm.rainbow(np.linspace(0, 1, 11))

# # Apply t-SNE to the features
# features_tsne = TSNE(n_components=2, init='pca', random_state=42).fit_transform(features)

# # Plot the t-SNE visualization
# plt.figure(figsize=(10, 8))
# for label in np.unique(labels):
#     plt.scatter(features_tsne[labels == label, 0], features_tsne[labels == label, 1], label=label, s=5)
# plt.legend()
# plt.show()